# Vacation and weekend call schedule solver 

## Assign vacations 

### 1. Import packages, upload input files, set weights and wages

In [326]:
import csv
import pandas as pd
from datetime import datetime, timedelta
from dplython import * 
from ortools.sat.python import cp_model

basedir = "C:/Users/mitch/Documents/OHSU/Chief Stuff/Schedule Stuff/Final pre-switch assignments"
sys.path.append(basedir)

from general import *

requests = basedir + "/test.csv"
rotations = basedir + "/rotations.csv"

# reading in file with list of requests 
# there are three columns: name, request, date 
req = pd.read_csv(requests, encoding= 'unicode_escape') 
rot = pd.read_csv(rotations, encoding= 'unicode_escape') 

# identify unique resident names
res = rot.name.drop_duplicates().values.tolist()
num_res = len(res)

#specify PGY level based on name 
res_level = rot.loc[:,['name','pgy']].drop_duplicates().pgy.values.tolist()

#weights
#fixed weight and unavailable weight used for blackout weekend reqeusts as well
fixed_weight = 100000
soft_vaca_weight = 200
priority_vaca_weight = 100 
backup_vaca_weight = 50 
unavailable_weight = -100000

# identify unique rotations
rotation = rot.groupby('rotation').groups.keys()

off_hill_rotations = ['kaiser','transplant','research']
off_hill_wknds = ['kaiser','transplant']
hill_rotations = [rota for rota in rotation if rota not in off_hill_rotations]

# specify the PGY request 'pay raise' (in other words, specify the strength of the hierarchy)
pgyraise = 0.1

#number of max/min calls by PGY 
pgy = [0,0,0,1,1,1,2,2,2,3,3,3]
min_shifts = [12,7,8,8] # total
min_hol = [2,1,1,1] #holidays min
max_hol = [2,1,2,1] #holidays max
min_fri = [8,3,2,1] #Friday min
max_fri = [8,3,3,1] #Friday max 


In [327]:
#Special dates

# specify block dates
blockdates = [[datetime(2022,7,1),datetime(2022,10,31)],
             [datetime(2022,10,31),datetime(2023,2,28)],
             [datetime(2023,2,28),datetime(2023,6,30)]]  

# July 
july_mondays = [datetime(2022,7,4), 
               datetime(2022,7,11),
               datetime(2022,7,18),
               datetime(2022,7,25)]

# Last week in June
last_week = [datetime(2023,6,26)]

#specify conference dates that are blocked (monday of the week that is blocked)
# WSAUA (one week), AUA (two weeks)
    
conference_mondays = [datetime(2022,10,31), datetime(2023,4,24),datetime(2023,5,1)]
conference_fridays = [datetime(2022,8,12),datetime(2022,10,28),datetime(2022,11,4), datetime(2023,4,28)]

#graduation weekend
graduation_friday = [datetime(2023,6,16)]
    
# Holidays (usually 8 per year, but this year, June 30th is probably July 4th weekend for the following year so will treat as such = 9 holidays total)
# July 4th, Labor 9/5, Thanksgiving, Xmas, NYE, MLK 1/16, Prez 2/20, Memorial 5/29, July 4th the next year 
# (Sa,Mo), (Sa,Mo), (Th,Sa), (Su), (Su), (Sa/Mon), (Sa/Mon), (Sa/Mo), (Sa/Mon)
holiday_fridays = [datetime(2022,7,1),
                  datetime(2022,9,2),
                  datetime(2022,11,25),
                  datetime(2022,12,23),
                  datetime(2022,12,30),
                  datetime(2023,1,13),
                  datetime(2023,2,17),
                  datetime(2023,5,26), 
                  datetime(2023,6,30)]

senior_holidays = [datetime(2022,11,25),
                  datetime(2022,12,23),
                  datetime(2022,12,30)]

#Total - Fridays = Saturdays duh!


In [328]:
#Universally unavailable vacation weeks
#The last week in June can be taken for PGY5s and this is hard coded into the schedule after the other assignments are made
unavailable_weeks = july_mondays + last_week + conference_mondays 

# use get_dates function to get dates starting last week to today
start_date = '2022-07-01'
end_date = (datetime.strptime(start_date,"%Y-%m-%d") + timedelta(days=364)).strftime("%Y-%m-%d")
date_list = get_dates(start_date, end_date)

#create a list of lists representing [index, datetime object, date string] for specific weekdays during the year
mondays = get_weekday_list('Monday', date_list)
fridays = get_weekday_list('Friday',date_list)
saturdays = get_weekday_list('Saturday',date_list)

#if July 1st is a Saturday then drop it 
#if June 30th is a Friday, then add July 1st of the next year to Saturdays
if saturdays[0] == datetime.strptime(start_date,"%Y-%m-%d"):
    del saturdays[0]
if fridays[-1]== datetime.strptime(end_date,"%Y-%m-%d"):
    saturdays.append(datetime.strptime(end_date,"%Y-%m-%d")+timedelta(days=1))

In [329]:
# create a combined list of all Mondays, Fridays, Saturdays in the year 
monfrisat = combine_sort([mondays,fridays,saturdays])
    
#create a list of Fridays and Saturdays (aka weekend call shifts)
shiftlist = combine_sort([fridays,saturdays])

#create a list of Fridays and Saturdays that are holiday shifts 
holidayshifts = [x for x in shiftlist if x in holiday_fridays or x-timedelta(days=1) in holiday_fridays] 

#create a list of Fridays and Saturdays that are not holiday shifts 
nonholiday_fridays =  [x for x in fridays if x not in holidayshifts]
nonholidayshifts = [x for x in shiftlist if x not in holidayshifts]

In [330]:
colnames = []
for d in monfrisat: 
    colnames.append(name(d))

mastersched = pd.DataFrame(columns=colnames,index=res)

### 2. Make dictionaries of requests and rotation dates 


In [331]:
# create a dictionary of dictionaries, 
# the key in the first level is resident name, 
# the keys in the second level are the type of request

request_dict = make_request_dict(req, res)

rotation_dict = make_rotation_dict(rot, res)

#### Final check to see if anyone requested a weekend they weren't supposed to 

In [332]:
peds_reqs = 0 
va_reqs = 0 
uni_reqs = 0

req_made = []

for n, r in enumerate(res): 
    firstvacareqs= request_dict[r]['vaca1']+request_dict[r]['bovaca']+request_dict[r]['sfvaca']
    vacareqs = firstvacareqs +request_dict[r]['vaca2']
    wkndreqs = request_dict[r]['wknd']+request_dict[r]['bowknd']
    res_peds_reqs = 0 
    res_va_reqs = 0 
    res_uni1_reqs = 0
    res_uni2_reqs = 0
    for m in mondays: 
        for o in hill_rotations: 
            if m in vacareqs: 
                if m in july_mondays + conference_mondays and len(rotation_dict[r][o])==2 and rotation_dict[r][o][0] <= m <=  rotation_dict[r][o][1]:
                    print(r, "requested this unavailable week: ", name(m))
                for s,e in blockdates:
                    if e - timedelta(days=1) <= m <= e: 
                        print(r, "requested this border week off", name(m))
            if m in firstvacareqs and len(rotation_dict[r][o])==2 and rotation_dict[r][o][0] - timedelta(days=2) <= m <=  rotation_dict[r][o][1]- timedelta(days=2):
                if o == 'peds':
                    peds_reqs = peds_reqs +1
                    res_peds_reqs = res_peds_reqs +1
                if o in ['va', 'va1', 'va2']:
                    va_reqs = va_reqs + 1
                    res_va_reqs = res_va_reqs +1
                if o in ['uni','uni1']:
                    uni_reqs = uni_reqs +1
                    res_uni1_reqs = res_uni1_reqs +1 
                if o == 'uni2':
                    uni_reqs = uni_reqs +1
                    res_uni2_reqs = res_uni2_reqs +1 
    if res_uni1_reqs ==2: 
        print(r, " is taking 2 weeks on Uni")
    if res_uni2_reqs ==2: 
        print(r, " is taking 2 weeks on Uni")
    if res_peds_reqs ==2: 
        print(r, " is taking 2 weeks on Peds")
    if res_va_reqs ==2: 
        print(r, " is taking 2 weeks on VA")

    if res_level[n]==5 and last_week[0] in firstvacareqs: 
        if rotation_dict[r]['uni2'][0] <= last_week[0] < rotation_dict[r]['uni2'][1]: 
            res_uni2_reqs = res_uni2_reqs -1
        if rotation_dict[r]['va'][0] <= last_week[0] < rotation_dict[r]['va'][1]: 
            res_va_reqs = res_va_reqs -1

    req_made.append([res_uni1_reqs,res_uni2_reqs,res_va_reqs,res_peds_reqs])

    for f in fridays: 
        if f in wkndreqs and f in conference_fridays:  
            for o in hill_rotations: 
                if len(rotation_dict[r][o])==2 and rotation_dict[r][o][0] <= f <=  rotation_dict[r][o][1]:
                    print(r, "requested this conference weekend off: ", name(f))



# req_made is an array of length num_res, each element length number of hill rotations (4 = uni1, uni2, va, peds)   
# each number represents the number of requests made for each rotation, minus the last week in June for chiefs
print("")
print("Total uni first requests:", uni_reqs)
print("Total VA first requests:", va_reqs)
print("Total peds first requests:", peds_reqs)

Keiko  is taking 2 weeks on Uni
Lauren  is taking 2 weeks on VA
Jack  is taking 2 weeks on Peds
Jack requested this conference weekend off:  2022-08-12
Emily  is taking 2 weeks on VA
MLB  is taking 2 weeks on Uni
Amir  is taking 2 weeks on Peds
Jasper  is taking 2 weeks on Uni
Mitch  is taking 2 weeks on VA
Alex  is taking 2 weeks on Uni

Total uni first requests: 16
Total VA first requests: 12
Total peds first requests: 8


### 3. Make a table of week off requests 

#### First part creates a list of Mondays, Fridays, and Saturdays during the year 

#### This table will introduce multiple constraints via the 'cost' matrix: 
##### A. If they are not on the hill, they are taken out of the vacation pool 
##### B. Universally unavailable dates defined above

#### This section also counts the number of on-hill requests made for each block. The last week in June for PGY5s does not count because this will be hard coded into the schedule later. 

In [333]:
# create empty dataframe where resident name is the index
wks = pd.DataFrame({'resident': res})
wks = wks.set_index('resident')

for m in mondays:
    wks[name(m)]=0
    for r in res:
        if m in request_dict[r]['bovaca']: 
            wks.loc[r][name(m)]= fixed_weight
        elif m in request_dict[r]['sfvaca']: 
            wks.loc[r][name(m)]= soft_vaca_weight
        elif m in request_dict[r]['vaca1']: 
            wks.loc[r][name(m)]= priority_vaca_weight
        elif m in request_dict[r]['vaca2']:
            wks.loc[r][name(m)]= backup_vaca_weight
        else: 
            wks.loc[r][name(m)]= 0

# multiply each request by the PGY pay raise 
wksr = pd.DataFrame({'resident': res,
                     'res_level': res_level})
wksr = wksr.set_index('resident')
wksr = wksr >> mutate(res_level_mod = 1+(X.res_level-2)*pgyraise)


#make resident unavailable for vacation solver if off hill (requests only inputted into solver if on hill)
for n, r in enumerate(res):
    for m in mondays:        
        for o in off_hill_rotations:
            if len(rotation_dict[r][o])==2 and rotation_dict[r][o][0]-timedelta(days=2) <= m <=  rotation_dict[r][o][1] -timedelta(days=2):
                wks.loc[r, name(m)] = unavailable_weight

#makes PGY level adjustments
#block out unavailable weeks          
for m in mondays: 
    wks[name(m)]=wksr['res_level_mod']*wks[name(m)]
    if m in unavailable_weeks:
        wks[name(m)] = unavailable_weight
        
wks = pd.concat([wksr,wks], axis = 1)


In [334]:
rotation_weeks =[]
rotations = ['uni1','uni2','va','peds']
for n, r in enumerate(res): 
    uni1_weeks =  []
    uni2_weeks = []
    va_weeks = []
    peds_weeks = []
    for i,m in enumerate(mondays):
        if len(rotation_dict[r]['uni'])==2 and rotation_dict[r]['uni'][0]-timedelta(days=2) <= m < rotation_dict[r]['uni'][1]-timedelta(days=2):
            uni1_weeks.append(i)
        if len(rotation_dict[r]['uni1'])==2 and rotation_dict[r]['uni1'][0]-timedelta(days=2) <= m < rotation_dict[r]['uni1'][1]-timedelta(days=2):
            uni1_weeks.append(i)
        if len(rotation_dict[r]['uni2'])==2 and rotation_dict[r]['uni2'][0]-timedelta(days=2) <= m < rotation_dict[r]['uni2'][1]-timedelta(days=2):
            uni2_weeks.append(i)
        if len(rotation_dict[r]['va'])==2 and rotation_dict[r]['va'][0]-timedelta(days=2) <= m < rotation_dict[r]['va'][1]-timedelta(days=2):
            va_weeks.append(i)
        if len(rotation_dict[r]['va1'])==2 and rotation_dict[r]['va1'][0]-timedelta(days=2) <= m < rotation_dict[r]['va1'][1]-timedelta(days=2):
            va_weeks.append(i)
        if len(rotation_dict[r]['va2'])==2 and rotation_dict[r]['va2'][0]-timedelta(days=2) <= m < rotation_dict[r]['va2'][1]-timedelta(days=2):
            va_weeks.append(i)
        if len(rotation_dict[r]['peds'])==2 and rotation_dict[r]['peds'][0]-timedelta(days=2) <= m < rotation_dict[r]['peds'][1]-timedelta(days=2):
            peds_weeks.append(i)
    rotation_weeks.append([uni1_weeks,uni2_weeks,va_weeks,peds_weeks])

### 5. Invoke the vacation scheduling solver 

In [335]:
# for each block, 
# assign vacations 

mondaynames = []
for m in mondays:
    mondaynames.append(name(m))
requests = wks[mondaynames].values.tolist()
num_weeks = len(requests[0])
all_wks = range(num_weeks)
all_res = range(num_res)

model = cp_model.CpModel()

vaca = {}
for r in range(num_res):
    for week in range(num_weeks):
        vaca[r, week] = model.NewBoolVar(f'vaca[{r},{week}]')


# No two vacations in a row 
# the total number of requests is equal to the requests made 
for r in range(num_res):
    for week in range(num_weeks-1):
        model.Add(vaca[r,week]+vaca[r,week+1] != 2)


# individual resident service requests 
service_vacas = []
for n, s in enumerate(rotations): 
    one_service_vaca = []
    for r in all_res:
        tmp = []
        for week in rotation_weeks[r][n]:
            tmp.append(vaca[r,week])
            one_service_vaca.append(vaca[r,week])
        model.Add(sum(tmp) == req_made[r][n]) 
    service_vacas.append(one_service_vaca)

#all resident service caps 
model.Add(sum(service_vacas[0]+service_vacas[1]) == np.sum(req_made, axis=0)[0] + np.sum(req_made, axis=0)[1])
model.Add(sum(service_vacas[2]) == np.sum(req_made, axis=0)[2] )
model.Add(sum(service_vacas[3]) == np.sum(req_made, axis=0)[3] )
    
    
# No more than 1 vacation per week
for week in range(num_weeks):
    model.Add(sum(vaca[r,week] for r in range(num_res)) <= 1)

# create a 'cost' matrix, that will be used to optimize the vacation requests 
pref = []
for r in range(num_res):
    for week in range(num_weeks):
        pref.append(requests[r][week]* vaca[r,week])

model.Maximize(sum(pref))

solver = cp_model.CpSolver()
status = solver.Solve(model)

vacasched = pd.DataFrame(columns=mondaynames,index=res)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Total preference points = {solver.ObjectiveValue()}\n')
    top_req = 0 
    sec_req = 0
    non_req = 0
    for r in range(num_res):
        for week in range(num_weeks):
            if solver.BooleanValue(vaca[r, week]):
                if wks.loc[res[r],name(mondays[week])]<0: 
                    print(res[r],'assigned to',name(mondays[week]),'but not available')
                if wks.loc[res[r],name(mondays[week])]==0: 
                    print(res[r],'assigned to',name(mondays[week]),'but not requested')
                    non_req = non_req +1
                if wks.loc[res[r],name(mondays[week])]>= priority_vaca_weight:
                    top_req = top_req +1 
                if priority_vaca_weight > wks.loc[res[r],name(mondays[week])]>= backup_vaca_weight:
                    sec_req = sec_req +1
                vacasched.loc[res[r],name(mondays[week])]=1
    
    print(top_req,'first requests assigned')
    print(sec_req,'second requests assigned')
    print(non_req,'non requests assigned')
else:
    print('No solution found.')


    

Total preference points = 342705.0

Keiko assigned to 2023-06-19 but not requested
Lauren assigned to 2023-06-12 but not requested
Jack assigned to 2023-01-09 but not requested
MLB assigned to 2022-10-24 but not requested
Amir assigned to 2022-12-12 but not requested
Amir assigned to 2023-05-29 but not requested
24 first requests assigned
3 second requests assigned
6 non requests assigned


In [336]:
# Add back off hill rotation vacations, and PGY5 last week in June to check everyone getting 4 vacas

for n, r in enumerate(res):
    vacareqs = request_dict[r]['vaca1']+request_dict[r]['bovaca']
    for m in mondays:
        for o in off_hill_rotations:
            rotdates = rotation_dict[r][o]
            if len(rotdates)==2 and rotdates[0] <= m <=  rotdates[1] and m in vacareqs:
                vacasched.loc[r, name(m)] = 1
    if res_level[n] == 5 and datetime(2023,6,26) in vacareqs: 
        namejune = str(datetime(2023,6,26))[0:10]
        vacasched.loc[r, namejune] = 1

for n, r in enumerate(res):
    for m in mondays: 
        if vacasched.loc[r,name(m)]==1:
            mastersched.loc[r,name(m)]=1

            
vacasched

,2022-07-04,2022-07-11,2022-07-18,2022-07-25,2022-08-01,2022-08-08,2022-08-15,2022-08-22,2022-08-29,2022-09-05,...,2023-04-24,2023-05-01,2023-05-08,2023-05-15,2023-05-22,2023-05-29,2023-06-05,2023-06-12,2023-06-19,2023-06-26
Keiko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
Lauren,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
Jack,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Emily,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
Linh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
Jess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
MLB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
Amir,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
Craig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jasper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [337]:
wknds = vacasched

wknds= wknds.replace(to_replace = 1,
            value = -1)

wknds= wknds.fillna(0)

for f in fridays:
    s = f + timedelta(days=1)
    wknds[name(f)]= 0.0
    wknds[name(s)]= 0.0



for m in mondays: 
    wknds.pop(name(m))
wknds

C:\Users\mitch\AppData\Local\Temp/ipykernel_1648/493184118.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  wknds[name(s)]= 0.0


,2022-07-01,2022-07-02,2022-07-08,2022-07-09,2022-07-15,2022-07-16,2022-07-22,2022-07-23,2022-07-29,2022-07-30,...,2023-06-02,2023-06-03,2023-06-09,2023-06-10,2023-06-16,2023-06-17,2023-06-23,2023-06-24,2023-06-30,2023-07-01
Keiko,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lauren,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jack,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Emily,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Linh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jess,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MLB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Amir,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Craig,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jasper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Assign weekends 

### 1. Make a table of weekend requests and unavailabilities 

In [338]:
#borrow the vacation schedule to designate all vacation bookend weekends as unavailable
wknds = vacasched

wknds= wknds.replace(to_replace = 1,
            value = -1)

wknds= wknds.fillna(0)

for f in fridays:
    s = f + timedelta(days=1)
    wknds[name(f)]= 0.0
    wknds[name(s)]= 0.0

for n, r in enumerate(res): 
    birthday = request_dict[r]['birthday'][0]
    birthdayweekend = min([f for f in fridays], key=lambda x: abs(x - birthday))

    for m in mondays:                   #Flanking weekends to vacation weeks
        fb = m - timedelta(days=3)      #Friday before
        sb = m - timedelta(days=2)      #Saturday before
        fa = m + timedelta(days=4)      #Friday after
        sa = m + timedelta(days=5)      #Saturday after

        if wknds.loc[r][name(m)]==-1:
            wknds.loc[r][name(fb), name(sb), name(fa), name(sa)]= unavailable_weight

    for f in fridays:
        s = f + timedelta(days=1)

        #Unavailable constraints  
        ca = f in request_dict[r]['bowknd']                     #very important weekend requests
        cb = f < datetime(2022,9,1) and res_level[n] == 2       #PGY2 no call in first two months
        cc = f in conference_fridays and res_level[n] >= 4      #PGY4/5 no call for important conferences
        cd = f > datetime(2023,6,1) and res_level[n] == 5       #PGY5 no call in June 
        ce = f in senior_holidays and res_level[n] >= 4         #PGY4/5 no call on Thanksgiving, Xmas, New Years 
    
        if ca or cb or cc or cd or ce is True:
            wknds.loc[r][name(f),name(s)]= unavailable_weight
        
        #Soft constraints 
        ch = f in request_dict[r]['wknd']                       #weekend requests
        ci = f == birthdayweekend                               #birthday weekends

        if ch or ci is True: 
            wknds.loc[r][name(f),name(s)]= -1

        # off hill residents out of the pool
        for o in off_hill_wknds:
            if len(rotation_dict[r][o])==2 and rotation_dict[r][o][0] <= f <=  rotation_dict[r][o][1]:
                wknds.loc[r][name(f),name(s)]= unavailable_weight
            
#Now get rid of Monday dates in the table
for m in mondays: 
    wknds.pop(name(m))


C:\Users\mitch\AppData\Local\Temp/ipykernel_1648/4057225559.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  wknds[name(s)]= 0.0


In [339]:
#creates nested list of all weekend requests/availabilities
num_wknds = len(fridays)
all_wknds = range(num_wknds)
all_requests = []
for r in res:
    n = 0
    wkndreq = []
    for i in all_wknds:
        shiftreq = []
        shiftreq.append(wknds.loc[r][n])
        shiftreq.append(wknds.loc[r][n+1])
        n = n +2 
        wkndreq.append(shiftreq)
    all_requests.append(wkndreq)

 

In [340]:
num_wknds = len(fridays)
all_wknds = range(num_wknds)
num_shifts = 2
all_shifts = range(num_shifts)

#create the model
model = cp_model.CpModel()

#define the variables (two shifts per weekend)
shifts = {}
for r in all_res:
    for w in all_wknds:
        for s in all_shifts:
            shifts[(r, w,
                    s)] = model.NewBoolVar('shift_n%id%is%i' % (r, w, s))

#BASELINE CONSTRAINTS 
                     
#one resident per shift 
for w in all_wknds:
    for s in all_shifts:
        model.Add(sum(shifts[(r, w, s)] for r in all_res) == 1)

#one shift per weekend 
for r in all_res:
    for w in all_wknds:
        model.Add(sum(shifts[(r, w, s)] for s in all_shifts) <= 1)

#at least one week off
for n, r in enumerate(all_res):    
    for w in range(num_wknds-2):
        model.Add((sum(shifts[(r,w,s)] for s in all_shifts) + 
                  sum(shifts[(r,w+1,s)] for s in all_shifts) + 
                  sum(shifts[(r,w+2,s)] for s in all_shifts)) != 2) 
        
#shifts per resident
for r in all_res: 
    model.Add(sum(shifts[(r, w, s)] for s in all_shifts for w in all_wknds) >= min_shifts[pgy[r]])
    model.Add(sum(shifts[(r, w, s)] for s in all_shifts for w in [i for i,f in enumerate(fridays) if f in holiday_fridays]) >= min_hol[pgy[r]])
    model.Add(sum(shifts[(r, w, s)] for s in all_shifts for w in [i for i,f in enumerate(fridays) if f in holiday_fridays]) <= max_hol[pgy[r]])
    model.Add(sum(shifts[(r, w, s)] for s in [0] for w in [i for i,f in enumerate(fridays) if f not in holiday_fridays]) >= min_fri[pgy[r]])
    model.Add(sum(shifts[(r, w, s)] for s in [0] for w in [i for i,f in enumerate(fridays) if f not in holiday_fridays]) <= max_fri[pgy[r]])
    
#pair PGY2s with seniors before Jan 1 (except holidays and conferences)
for w in [wknds for wknds, f in enumerate(fridays) if f < datetime(2023,1,1) and f not in senior_holidays+conference_fridays]:
    pgyjunior = []
    for s in all_shifts:
        for r in [n for n, pgy in enumerate(res_level) if pgy < 4]:
            pgyjunior.append(shifts[(r,w,s)])
    model.Add(sum(pgyjunior)<=1)

#and for conferences before Jan 1, no more than one PGY2 per weekend
for w in [wknds for wknds, f in enumerate(fridays) if f < datetime(2023,1,1) and f in conference_fridays]:
    pgyjunior = []
    for s in all_shifts:
        for r in [n for n, pgy in enumerate(res_level) if pgy == 2]:
            pgyjunior.append(shifts[(r,w,s)])
    model.Add(sum(pgyjunior)<=1)

#solve the problem 
model.Maximize(
    sum(all_requests[r][w][s] * shifts[(r, w, s)] for r in all_res
        for w in all_wknds for s in all_shifts))

solver = cp_model.CpSolver()
status = solver.Solve(model)

allassign = pd.DataFrame(columns=[name(fs) for fs in shiftlist],index=res)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for w in all_wknds:
        for r in all_res:
            for s in all_shifts:
                if solver.Value(shifts[(r, w, s)]) == 1:
                    if all_requests[r][w][s] < -1:
                        print(res[r],"assigned to", [name(f) for f in fridays][w], 'but requested off or on vacation')
                    
                    allassign.loc[res[r],name([f for f in fridays][w]+s*timedelta(days=1))]=1

else:
    print('No optimal solution found !')   


In [341]:
#builds assignment dictionary to create individualized text file reports 

all_assignments = {}

va_vacations = 0
peds_vacations = 0 
uni_vacations = 0 

for r in res:
    res_assignments = {'holidays':[],
                      'other weekend shifts':[],
                      'vacation first requests granted':[],
                      'vacation second requests granted':[],
                      'non-request vacations assigned':[],
                      'weekend shift request off granted':[],
                      'weekend shift requested off but assigned':[]}
    for fs in [fs for fs in shiftlist if fs in holidayshifts]:
        if allassign.loc[r][name(fs)] ==1: 
            wknds.loc[r][name(fs)]=1
            mastersched.loc[r][name(fs)]=1
            if fs in fridays:
                res_assignments['holidays'].append(name(fs) + " Friday/Sunday shift")
            if fs in saturdays:
                res_assignments['holidays'].append(name(fs) + " (Thursday?)/Saturday/(Monday?) shift")
    for fs in [fs for fs in shiftlist if fs not in holidayshifts]: 
        if allassign.loc[r][name(fs)] ==1: 
            wknds.loc[r][name(fs)]=1
            mastersched.loc[r][name(fs)]=1
            if fs in fridays:
                res_assignments['other weekend shifts'].append(name(fs)+ " Friday/Sunday")
            if fs in saturdays:
                res_assignments['other weekend shifts'].append(name(fs)+ " Saturday")
    
    wkndreqs = request_dict[r]['wknd']+request_dict[r]['bowknd']
    firstvacareqs = request_dict[r]['vaca1']+request_dict[r]['bovaca']+request_dict[r]['sfvaca']
    
    for m in mondays: 
        if vacasched.loc[r][name(m)] ==1: 
            mastersched.loc[r][name(m)]=1
            if m in firstvacareqs:
                res_assignments['vacation first requests granted'].append(name(m))
            elif m in request_dict[r]['vaca2']:
                res_assignments['vacation second requests granted'].append(name(m))
            else: 
                res_assignments['non-request vacations assigned'].append(name(m))
    
    for f in fridays:
        if allassign.loc[r][name(f)] ==1 and f in wkndreqs:
            res_assignments['weekend shift requested off but assigned'].append(name)
        if allassign.loc[r][name(f)] !=1 and f in wkndreqs:
            res_assignments['weekend shift request off granted'].append(name)
        
    
    for m in mondays:
        if vacasched.loc[r][name(m)] ==1:
            if len(rotation_dict[r]['va1'])==2 and rotation_dict[r]['va1'][0] <= m < rotation_dict[r]['va1'][1]: 
                va_vacations = va_vacations +1
            if len(rotation_dict[r]['va'])==2 and rotation_dict[r]['va'][0] <= m < rotation_dict[r]['va'][1]: 
                va_vacations = va_vacations +1 
            if len(rotation_dict[r]['va2'])==2 and rotation_dict[r]['va2'][0] <= m < rotation_dict[r]['va2'][1]: 
                va_vacations = va_vacations +1 
            if len(rotation_dict[r]['peds'])==2 and rotation_dict[r]['peds'][0] <= m < rotation_dict[r]['peds'][1]: 
                peds_vacations = peds_vacations +1 
            if len(rotation_dict[r]['uni1'])==2 and rotation_dict[r]['uni1'][0] <= m < rotation_dict[r]['uni1'][1]: 
                uni_vacations = uni_vacations +1 
            if len(rotation_dict[r]['uni'])==2 and rotation_dict[r]['uni'][0] <= m < rotation_dict[r]['uni'][1]: 
                uni_vacations = uni_vacations +1 
            if len(rotation_dict[r]['uni2'])==2 and rotation_dict[r]['uni2'][0] <= m < rotation_dict[r]['uni2'][1]: 
                uni_vacations = uni_vacations +1 
            
        
    all_assignments[r]= res_assignments
            

In [342]:
#another check to make sure vacations spread out evenly 

print(va_vacations/3)
print(uni_vacations/4)
print(peds_vacations/2)

4.0
4.0
4.0


In [343]:
print('Total weekend shifts per resident:') 
print(mastersched[[name(fs) for fs in shiftlist]].sum(axis=1))
print(mastersched[[name(fs) for fs in shiftlist]].sum(axis=1).sum(axis=0))
print('Holidays per resident:')
print(mastersched[[name(fs) for fs in holidayshifts]].sum(axis=1))
print('Non holiday Friday/Sundays per resident:')
print(mastersched[[name(f) for f in fridays if f not in holidayshifts]].sum(axis=1))

Total weekend shifts per resident:
Keiko     12.0
Lauren    12.0
Jack      12.0
Emily      7.0
Linh       7.0
Jess       7.0
MLB        8.0
Amir       8.0
Craig      9.0
Jasper     8.0
Mitch      8.0
Alex       8.0
dtype: float64
106.0
Holidays per resident:
Keiko     2.0
Lauren    2.0
Jack      2.0
Emily     1.0
Linh      1.0
Jess      1.0
MLB       2.0
Amir      2.0
Craig     2.0
Jasper    1.0
Mitch     1.0
Alex      1.0
dtype: float64
Non holiday Friday/Sundays per resident:
Keiko     8.0
Lauren    8.0
Jack      8.0
Emily     3.0
Linh      3.0
Jess      3.0
MLB       3.0
Amir      2.0
Craig     3.0
Jasper    1.0
Mitch     1.0
Alex      1.0
dtype: float64


In [ ]:
colnames = []
for mfs in monfrisat: 
    if mfs in holidayshifts: 
        colnames.append(name(mfs) + " (HOLIDAY)")
    elif mfs in fridays:
        colnames.append(name(mfs) + " (Friday)")
    elif mfs in saturdays:
        colnames.append(name(mfs) + " (Saturday)")
    else: 
        colnames.append(name(mfs) + " (Vacation)")

mastersched.columns=colnames

mastersched= mastersched.fillna('')

mastersched.to_excel('Master Schedule.xlsx', index=True)

wknds= wknds.replace(to_replace = unavailable_weight,
            value = 0)

wknds.to_excel('Weekend Schedule.xlsx', index=True)

vacasched= vacasched.fillna(0)

vacasched.to_excel('Vacation Schedule.xlsx', index = True)


0     2
3     2
6     2
9     3
14    3
19    3
24    4
27    4
30    4
33    5
36    5
39    5
Name: pgy, dtype: int64

In [ ]:
for r in res: 
    file = open(str(r+"_assignments2.txt"),"w")
    for key, value in all_assignments[r].items():
        file.write('%s:%s\n' % (key, value))
    file.close()
